In [1]:
from gensim.models import FastText
import gensim
import pandas as pd

DATA_PATH = '/data/'
MODEL_PATH = DATA_PATH + 'models/ft1/'
SOURCE_PATH = DATA_PATH + 'captions/lem/'

cities = ['moscow', 'spb']#, 'nyc', 'london']
years = ['2016', '2017', '2018', '2019', '2020']
files = []
for city in cities:
    for year in years:
        files.append([city, year])

def csv_path(path, city, year):
    return path + city + '_posts_' + year + '.csv'

valid_langs = set(['__label__ru'])

USE_PRETRAINED_MODEL = True

In [4]:
class MyCorpus(object):
    iter = 0
    def __iter__(self):
        self.iter += 1
        for city in cities:
            for year in years:
                df = pd.read_csv(csv_path(SOURCE_PATH, city, year))
                df = df[df.lang.isin(valid_langs)]
                for s in df['caption']:
                    yield s
                              
            print(f'iter {self.iter} for {city} completed')
                
        print(f'\niter {self.iter} completed\n')

sentences = MyCorpus()

# dimension 300, with character n-grams of length 5, a window of size 5 and 10 negatives
# https://fasttext.cc/docs/en/crawl-vectors.html

if USE_PRETRAINED_MODEL:
    model =  gensim.models.fasttext.load_facebook_model(MODEL_PATH + 'cc.ru.300.bin.gz')
else:
    model = FastText(size=300, window=5, min_count=5, word_ngrams=5, negative=10, workers=35)
print('success')

success


In [ ]:
print('building vocab\n')
model.build_vocab(sentences=MyCorpus())
total_examples = model.corpus_count
print(f'\ncorpus count: {total_examples}\nend of building')

building vocab

iter 1 for moscow completed


In [ ]:
print('start training\n')
model.train(sentences=MyCorpus(), total_examples=total_examples, epochs=5)
print(f'\nend of training')

start training



In [ ]:
model.save(MODEL_PATH + 'mdl')

f = open(MODEL_PATH + 'description.txt', 'w')
f.write('date: 08.05\n')
f.write('params: size=300, min_count=1, window=5, negative=10, word_ngrams=5, epochs=5\n')
f.write(f'sourse: {SOURCE_PATH}\n')
f.write('cities: moscow, spb; only russian\n')
print('success')
f.close()

In [29]:
print(len(model.wv.vocab))

5313070
